### Subclassification on the propensity score

The idea is to create blocks of observations in the treatment and control group that have similar propensity scores. Then, take the difference in the outcome within those groups, and average the differences across groups to get an estimate of the ATE. In other words, we assume that there as a block-randomized experiment where treatment assignment was random within each block.

How do we find the blocks? Imbens/Rubin (chapter 17.3) propose the following algorithm
1. Start with a single block, with boundaries equal to 0 and 1
2. Compute 2 statistics to decide whether the block should be further splitted or not:
    - Compute t-statistic testing the H0 that the average value of the propensity score is the same in treatment and control groups. 
    - Compute the number of observations in treatment and control groups that would remain in two new blocks created by splitting the current block at the median of the propensity score 
3. While t-statistic > threshold and new block sizes sufficiently large:
    - Split the block
    - Update the statistics above 


We need to define the following parameters
- t_max = 1.96
- the minimum number of treated or control units in a new block
- the minimum number of units (T + C) in a new block


**NOTE** the code here is not working yet. need to load data as in the other notebook, and import the modules

In [1]:
t_max = 1.96 
N_min1 = 3 
N_min2 = 2

In [ ]:
# TODO: load data here, following the other notebook

In [ ]:
nsw_stacked.head()
nsw_stacked.describe()

In [ ]:
# we'll use the linearized propensity score instead 
# nsw_stacked["lps"] = np.log(nsw_stacked["pscore"])
data = nsw_stacked.loc[:, ["treat", "lps"]].to_numpy()
b = np.ones((data.shape[0],1))
data = np.c_[data, b]

# compute the t-stat
W = data[:, 0] #treatment indicator

def summarise(a, block, W):
    """In a block for group W, compute:
    - the average propensity score
    - the number of observations
    - the within-group variance
    """
    # TODO: there are assumptions on which columns are which variable
    # perhaps just use vectors directly?

    N = np.sum(block * W)
    l = 1 / N * np.transpose(a[:, 1] * block).dot(W)
    sum_of_squared_diff = np.sum((a[:, 2] * block - l)**2)

    return l, N, sum_of_squared_diff

l_1, N_1, s2_1 = summarise(data, block=data[:, 2], W=W)
l_0, N_0, s2_0 = summarise(data, block=data[:, 2], W=np.ones(W.shape) - W)
s_hat = 1 / (N_1 + N_0 - 2) * (s2_0 + s2_1)

t_stat = (l_1 - l_0) / (np.sqrt(s_hat * (1 / N_1 + 1 / N_0)))

# count number of observation in a block if split by the median propensity score 
mid = np.median(data[:, 1])
block_a = data[:, 1] <= mid 
block_b = data[:, 1] > mid 

np.sum(data[:, 2] * block_a * W )
np.sum(data[:, 2] * block_a * (np.ones(W.shape) - W) )

np.sum(data[:, 2] * block_b * W )
# np.sum(data[:, 2] * block_b * (np.ones(W.shape) - W) )


In [ ]:
# # matching without replacement -- working code
# matched = []
# for idx_treat in range(N_treat):
#     distance = (treated[idx_treat, 1] - controls[:, 1])**2
#     idx_min = np.argmin(distance) # index of the min 
#     idx_ctrl = controls[idx_min][0] # index of the matched control
#     # update the pool of controls: drop the matched unit
#     mask = np.full(distance.shape, True)
#     mask[idx_min] = False
#     controls = controls[mask, ...]
#     # add the matched control index to matched_controls
#     matched.append((int(idx_ctrl), idx_treat))